# Q-learning using tables
##### Authors: Eirik Fagtun Kjærnli and Fabian Dietrichson

## Welcome 
Welcome to this workshop!

The workshop is structured such that for each cell, you will write your own code and the code will then be asserted in the next cell. The assertion cell is marked "# Do not edit - Assertion cell #". <br>
The code should be written between "Write code below" and "Write code above". If your code does not pass the assertion, you will have to rewrite it before continuing.

### Task:
Implment the method multiply_input_by_2
- The code should take the input variabel "input_variabel", and multiply it by 2
- The answer should be stored in the variabel "result"
- When the method is implemented, run the cell.

Hotkey to run a cell: CTRL + ENTER

In [ ]:
def multiply_input_by_2(input_variabel):

    "Write code below" 
    result = input_variabel * 2
    
    "Write code above" 
    
    return result

In [ ]:
# Do not edit - Assertion cell #
assert(multiply_input_by_2(10) == 20), "Your method did not multiple the input by 2"
print("Great, you correctly implemented the method!")

## Import necessary packages
To implement the native Q-learning algorithm and use it in an environment, we just need two additional packages.

_Numpy_
- Numpy adds support for large, multi-dimensional arrays and matrices, along with a large collection of high-level mathematical functions to operate on these arrays. We could do this workshop using native Python arrays and built-in methods, however using Numpy simplifies the process. 
- To call the methods in the package, we use the "as np" command. Such that, to create a numpy table with 2 rows and 2 columns where all the cells are zero, you can simply write np.seros(2,2).

_OpenAI Gym_
- The Gym library is a collection of demo problems an associated environment, that you can use to work out your reinforcement learning algorithms. These range from simple text problems, to complex physical problems, to Atari video games. OpenAI Gym is therefore by many the preferred framework to learn and test Reinforcement learning algorithms."


### Task:
To import the necessary packages, simply run the cell below, and then run the assertion cell to verify that they have been imported correctly.

In [ ]:
import numpy as np
import gym

In [ ]:
# Do not edit - Assertion cell #
try:
    assert(gym)
    print("Great, the packages were imported correctly!")
except:
    print("You did not run the cell above, do this before you continue!")

## Problem Introduction

The environment we are going to use is a simple grid world, where the agent is controlling a taxi. The environment was introduced in [Dietterich2000] to illustrate some issues in hierarchical reinforcement learning. There are 4 locations (labeled by different letters) and your job is to pick up the passenger at one location and drop the passenger off in another. You can read more about the environment here: https://gym.openai.com/envs/Taxi-v2/

### Task:
Create the environment variable containing all necessary methods to run the Taxi-v2 game. <br>

_Tip: Just run the cell below_

In [ ]:
environment_id = "Taxi-v2"
env = gym.make(environment_id)

In [ ]:
# Do not edit - Assertion cell #
try:
    assert(env)
    print("You successfully created the environement {}".format(env.spec.id))
except:
    print("You create the environment incorrectly!")

## Build support methods for workshop
Before you can go on, the cell below must be run. These are methods used to verify your work, in addition to the support function that will be used throughout the workshop. 

### Task
Do as you have done before, simply mark the cell below and press CTRL + Enter!

In [ ]:
from IPython.display import clear_output
import gym
import os
import time
from copy import deepcopy
import seaborn as sb
import matplotlib.pyplot as plt

class TestEnvironment():
    
    def __init__(self):
        self.env = gym.make(environment_id)
        self.env.reset()
        
    def play(self, action):
        if not (0 <= int(action) <= 5):
            print("Action value must either be 0,1,2,3,4 or 5")
            return

        clear_output()
        _, reward, done, _ = self.env.step(action)
        self.env.render()
        print("Reward: ", reward)
        
        if(done):
            print("Game completed, resetting environment!")
            self.reset_env()
        
    def reset_env(self):
        self.env.reset()
        print("Environment has been reset")
        time.sleep(2)
        clear_output()

class MockData():
    def __init__(self):
        self.env = gym.make(environment_id)
        self.env.seed(10)
        self.should_assert = True
        
    def get_Q(self):
        Q = np.zeros([self.env.observation_space.n, self.env.action_space.n])
        Q[0:2,0] = 10
        return Q
    
    def get_env(self):
        return self.env
    
    def turn_off_assertion(self):
        self.should_assert = False
        
    def do_assertion(self):
        return self.should_assert

def visualize_q_table():
    fig=plt.figure(figsize=(10, 10))
    heat_map = sb.heatmap(Q_trained)
    plt.show()

def plot_visualization(data):
    
    reward_list = data[0]
    iteration_list = data[1]
    epsilon_list = data[2]
    
    episodes = range(len(reward_list))
    
    plt.figure(figsize=(10,10))
    ax = plt.subplot(311)
    ax.set_title("Reward")
    ax = plt.plot(episodes, reward_list)

    ax = plt.subplot(312)
    ax.set_title("Iterations per episode")
    ax.plot(episodes, iteration_list)
    
    ax = plt.subplot(313)
    ax.set_title("Epsilon")
    ax.set_xlabel("Episodes")
    ax.plot(episodes, epsilon_list)
    
    plt.show()

def reset_env_and_update_params(env):
    done = False
    state = env.reset()
    iterations = 0
    total_reward = 0
    
    return state, done, iterations, total_reward

def render_performance(env, Q):
    iterations = 0
    total_reward = 0
    t_sleep = 1.2
    
    state = env.reset()
    env.render()
    time.sleep(t_sleep)
    clear_output()
    done = False

    while not done:
        action = np.argmax(Q[state,:]) 

        new_state, reward, done, _ = env.step(action)
        state =  new_state

        env.render()
        time.sleep(t_sleep)
        clear_output()
        
        iterations += 1
        total_reward += reward
        
        if (iterations >= 20): 
            done = True
            print("Agent did not complete the episode within 20 iterations, train your agent better!")
            return
    
    print("Your agent completed the task using {} iterations, \
          and got a total reward of {}".format(iterations, total_reward))
    
test_env = TestEnvironment()
mock = MockData()

In [ ]:
# Do not edit - Assertion cell #
try:
    assert(mock)
    print("Great, the support methods were created!")
except:
    print("You did not run the cell above, do this before you continue!")

## Experiment with the environment
Before we start creating the Q-learning agent, we will explore the environment we are going to use. This is done using the support methods we have created for you.

**How to play**
1. Simply input an action, e.g. the integer 1, where it is indicated in the cell below, and click CTRL + Enter. 
2. The game will reset when you have picked up and then dropped off the passenger at the indicated location.
3. If you would like to reset the test environment during the execution, run the cell which indicates this below. 

*Tip*: The bold colored letter shows where the passenger should be picked up, and the normal colored letter shows the drop-off spot.

**Possible actions**
0. Move down
1. Move up
2. Move right
3. Move left
4. Pick up passenger
5. Drop off passenger

### Task:
Play around with the environment to understand the rewards and how the game dynamics work. When you feel confident with the environment,  move on to the next cell.

In [ ]:
# Run this to reset the environment
test_env.reset_env()

In [ ]:
# Run this cell to play the game

# Input your action below
action = 0
# Input your action above

test_env.play(action)

## Discussion point
The game above is a simple environment, which is easy for humans to solve. However, if you were to use traditional programming, how would you solve it? 

### Task
Use 5 minutes in the group to come up with a strategy, and create a quick draft!

<img src="Images/Discussion.jpg" alt="drawing" width="400" height="200"/>

# Q-learning

In Reinforcement learning an agent moves from a state $S_{t}$, to a new state $S_{t+1}$ by taking an action $A_{t}$, and receiving the reward $R_{t}$ as illustrated below. The agent will repeat this process for a defined amount of iterations, and this process as a whole is known as an episode. The goal of the Reinforcement learning agent is to maximize the total reward it can collect during an episode. To improve the performance, it learns from each state transition, i.e. move from $S_{t}$ to $S_{t+1}$. How it learns, is what separates the different Reinforcement learning algorithms.

<img src="Images/Agent.png" alt="drawing" width="500" height="500"/>

In this notebook, we will work with one of the most iconic Reinforcement learning algorithms in its simplest form, namely Q-learning using Q-tables. The Q-tables holds the values of the strategy that gives us the highest reward,  according to the agent's knowledge. The table can be thought of as the brain of the agent. Each row in the table represents a state, and the columns represent the possible actions in that state. This will be explained further in the next section.

A Q-value is a measure of the total expected reward the agent will receive if it chooses that action and always picks the action with the highest Q-value in the succeeding states, based on its current knowledge of the environment, i.e. the Q-table. Simply said, the higher the Q-value, the better the agent believes the action is.

The Q-learning algorithm is used to update the Q-values for a given action $A_{t}$ in the state $S_{t}$, and is updated after each iteration. The equation is shown below.

$
\begin{equation}
Q(s_{t},a_{t})^{new} \leftarrow Q(s_{t}, a_{t}) + \alpha \ \big[r_{t} + \gamma \ \underset{a}{max} \  Q(s_{t+1},a) - Q(s_{t},a_{t})\big]
\end{equation}
$

A key point to note in this equation is that we are updating the Q(s, a) by using the highest Q-value in the next state, $S_{t+1}$. This value is unknown, and it is therefore just the agent's estimate of that state. This makes it an optimization problem were we gradually adjust each Q-value as we receive a reward moving between the states.


This might seem daunting at first, however, we will through this notebook break it into simple pieces, which will hopefully, make it easier to grasp.

## Create the Q-table
The Q-table is the brain of the agent and stores the agent's current knowledge of the Q-values of each state. Each row represents a state, and the columns represent all possible actions in that state. As a result, our table will have the dimensions, (rows=number of states, columns=number of actions). An illustration of the table is shown below, although without the labels on the columns. 

<img src="Images/Taxi_matrix_initial.png" alt="drawing" width="500" height="500"/>

### Task:
Create and return a Q-table using numpy, where each cell is initialized to zeros:
- Create the Q-table by using the numpy command: np.zeros(rows, columns). <br>
    https://docs.scipy.org/doc/numpy/reference/generated/numpy.zeros.html
- Use the following command to get the number of states in the environment:
    - env.observation_space.n
- Use the following command to get the number of possible actions. PS. all states have the same amount of actions.
    - env.action_space.n

In [ ]:
def create_q_table(env):
    
    "Write code below" 
    action_size = env.action_space.n
    observation_size = env.observation_space.n
    
    Q_table = np.zeros([env.observation_space.n, env.action_space.n])
    
    "Write code above" 
    
    return Q_table

In [ ]:
# Do not edit - Assertion cell #
if mock.do_assertion():
    assert(np.count_nonzero(mock.get_Q() == 0)), "All values in Q-table should be zero"
    assert(create_q_table(mock.get_env()).shape == (mock.get_env().observation_space.n, mock.get_env().action_space.n)), \
    "The dimensions are wrong"
    print("The Q-table was correctly built! " +
          "It has {} rows, each representing a unique state, and {} columns, each representing an action for that state."\
          .format(mock.get_env().observation_space.n, mock.get_env().action_space.n))

## Select the best action
Before we implement the Q-learning algorithm, we will implement a method that, given a state, chooses the action with the highest Q-value.

### Task
Return the column index of the highest Q-value in the Q-table, given a state.
- Tip: Use the argmax function from the numpy library. <br> https://docs.scipy.org/doc/numpy/reference/generated/numpy.argmax.html

In [ ]:
def get_best_action(Q_table, state):
    
    "Write code below" 
    best_action =  np.argmax(Q_table[state,:]) 
    
    "Write code above"
    
    return best_action

In [ ]:
# Do not edit - Assertion cell #
if mock.do_assertion():
    assert(get_best_action(mock.get_Q(), 1) == 0), "The method did not pick the action with the highest Q_value"
    print("The best action for the test state was chosen, excellent!")

## Select an action

Initially, all the Q-values in the Q-table are set to zero. As we begin to move around the environment we can end up in a loop, such that we only explore a small part of the environment. This problem is so common its gotten a name; The Exploration vs Exploitation dilemma!

One simple and very effective solution is to use a strategy called Epsilon-Greedy. The strategy makes the agent choose mostly random actions in the initial episodes,  which forces the agent to explore the environment. The strategy then gradually shifts the agent behavior towards making greedy choices, i.e. choosing the action with the highest Q-value. The parameter deciding the degree of random actions taken is called epsilon.

### Task:
Create a method that randomly picks an action IF a random value in the range [0,1] is less than epsilon, or ELSE picks the best action using the method we created earlier.   
- To pick a random action, use env.action_space.sample()
- To generate a random number with uniform probability in the range [0,1], use np.random.uniform()

In [ ]:
def select_action(Q_table, state, epsilon):
    
    "Write code below"
    if epsilon > np.random.uniform():
        action = env.action_space.sample()
    else:
        action = get_best_action(Q_table, state)
    
    "Write code above"    
    
    return action

In [ ]:
# Do not edit - Assertion cell #
if mock.do_assertion():
    assert(select_action(mock.get_Q(), 1, 0) == 0), \
    "Method should always return the same value for this state, since epsilon is 0"
    assert not (len(set([select_action(mock.get_Q(), 1, 1) for x in range(20)])) <= 2), \
    "Method should not return identical values when a random action should be chosen"
    print("The correct actions were picked, great job!")

## Gradually shift towards exploitation and reduce exploration of state-space

To make sure we are gradually moving from exploration to exploitation, we need to reduce the possibility of choosing a random action. In other words, we need to reduce epsilon. This is done by multiplying it with a constant called epsilon decay.

### Task:
Create a method which reduces epsilon by multiplying itself with the defined constant epsilon decay.

In [ ]:
def update_epsilon(epsilon):
    epsilon_decay = 0.95
    
    "Write code below" 
    epsilon *= epsilon_decay
    "Write code above"
    
    return epsilon

In [ ]:
# Do not edit - Assertion cell #
if mock.do_assertion():
    assert(update_epsilon(5) == 4.75), "Given an input of 5, the output should have been 4.75"
    print("Epsilon was correctly updated!")

 ## Find the highest Q_value for a given state

Now it is finally time to begin building our Q-learning algorithm. The method we are going to create first is the part marked red in the equation below. This method returns the highest Q-value of the new state.

$
\begin{equation}
Q(s_{t},a_{t})^{new} \leftarrow Q(s_{t}, a_{t}) + \alpha \ \big[r_{t} + \gamma \ {\color{red}{\underset{a}{max} \  Q(s_{t+1},a)}} - Q(s_{t},a_{t})\big]
\end{equation}
$ 


### Task
Return the highest Q-value for a given state, which is here called new_state.
- Tip: Use the numpy.max function to retrieve the column (action) with the highest Q-value for the given row(state)

In [ ]:
def find_highest_Q_value_in_state(Q_table, new_state):
    "Write code below"
    best_Q_value = np.max(Q_table[new_state,:])
    "Write code above"
    
    return best_Q_value

In [ ]:
# Do not edit - Assertion cell #
if mock.do_assertion():
    assert(find_highest_Q_value_in_state(mock.get_Q(), 1) == 10), \
    "The method did not pick the action with the highest Q_value"
    print("Perfect, the method returned the highest Q-value for that state!")

## Calculate error  in Q-value
The next method we are going to implement calculates how much we need to adjust our Q-value based on the reward we receive from moving from state $S_{t}$ to state $S_{t+1}$. In the intro to the Q-learning section, we defined the Q-value as the assumed expected total reward the agent will receive if it moves from state $S_{t}$ taking action $A_{t}$, and then always picks the action with the highest Q-value throughout the episode using the current Q-table. As a result, the the correct Q-value $Q(S_{t}, A_{t})$ can be defined as

$
Q(s_{t}, a_{t}) = r_{t} + \underset{a}{max} \  Q(s_{t+1},a)
$

Assuming $\gamma$ is equal to one, we see that the statement marked red would become equal to 0, if our Q-value was perfect, thereby making no changes to the Q-value.

$
\begin{equation}
Q(s_{t},a_{t})^{new} \leftarrow Q(s_{t}, a_{t}) + \alpha \ \big[{\color{red}{r_{t} + \gamma \ \underset{a}{max} \  Q(s_{t+1},a) - Q(s_{t},a_{t})}}\big]
\end{equation}
$

The parameter $\gamma$ is connected to how the agent values immediate rewards compared to later rewards, however explanation of this is outside the scope of this notebook. 

### Task:
Implement the part of the equation mark red.
- Use the method we created in the previous section to find the highest_q_value_in_state
- Use brackets notation to select the correct cell i.e. Q_table[Desired_state, desired_action]

In [ ]:
def calculate_error_in_Q_value(Q_table, state, action, reward, new_state):
    
    discount_factor = 0.95
    
    "Write code below"
    highest_q_value_in_state = find_highest_Q_value_in_state(Q_table, new_state)
    
    error_in_Q_value = reward + discount_factor * highest_q_value_in_state - Q_table[state,action]
    
    "Write code above"
    
    return error_in_Q_value

In [ ]:
# Do not edit - Assertion cell #
if mock.do_assertion():
    assert(calculate_error_in_Q_value(mock.get_Q(), 0, 0, 10, 1) == 9.5), \
    "The method did not calculate the correct error value for the test sample"
    print("The error in Q-value have been calculated correctly!")

## Update Q-table

We are almost ready to complete our Q-learning algorithm. 

The only element not yet explained is the parameter $\alpha$. $\alpha$ is known as the learning rate and controls how quick we update the Q-table. For example, a learning rate of 0.1 indicates that we will only use 10% of the actual error calculated when updating the Q-table. This is because we calculate the Q-value by using the highest Q-value of the next state, which is just our best estimate so far and might be erroneous. If that state has not been visited yet, it would simply have the initialized value of the table, i.e. 0. By using the learning rate, we account for this uncertainty and get a less oscillating system which is proven to give better results. 

$
\begin{equation}
Q(s_{t},a_{t})^{new} \leftarrow {\color{red}{Q(s_{t}, a_{t}) + \alpha \ \big[r_{t} + \gamma \ \underset{a}{max} \  Q(s_{t+1},a) - Q(s_{t},a_{t})\big]}}
\end{equation}
$

To implement the algorithm there is one factor which needs to be accounted for. If the episode has ended due to a terminal state, the last reward received will be the Q-value of that state, since there is no possible next state. As a result, if the done parameter is true (an input to the method below), we set the Q-value of that state-action pair equal to the reward.

### Task
Create a method which either returns
 - The Q-value of that state-action pair equal to the reward, IF it was the final episode. (Done is true)
 - ELSE updates the Q-table by following the Q-learning algorithm, using the methods we have created.

In [ ]:
def update_q_table(Q_table, state, action, done, reward, new_state):
    
    learning_rate = 0.8
    
    "Write code below"
    if(done):
        Q_table[state, action] = reward
    else:
        Q_table[state, action] = Q_table[state, action] + \
        learning_rate * calculate_error_in_Q_value(Q_table, state, action, reward, new_state)
    "Write code above"
    
    return Q_table

In [ ]:
# Do not edit - Assertion cell #
if mock.do_assertion():
    assert(update_q_table(mock.get_Q(), 2, 1, 1, 10, 3)[2,1] == 10), \
    "The updated Q-value when simulation was done was incorrect"
    assert(update_q_table(mock.get_Q(), 0, 0, 0, 10, 1)[0,0] == 17.6), \
    "The updated Q-value when the episode was not done, was incorrect"
    print("Q-tabel have been correctly updated!")

## Take one step

Now we are ready to create a method which makes our agent make a move in the environment.

### Task
Create a method which selects the best action, passes this to our environment, and returns the action, reward, done new_state variables.
- To choose an action use the select_action method we created earlier
- To make a step in the environment, use this method env.step(action)
- env.step(action) return the following values accordingly: new_state, reward, done, state_transition_probability
- Ignore the state_transition_probability, as this is not used in this environment
    

In [ ]:
def step(Q_table, env, state, epsilon):
    
    "Write code below"
    action = select_action(Q_table, state, epsilon)
    new_state, reward, done, _ = env.step(action)
    
    "Write code above"
    
    return action, reward, done, new_state

In [ ]:
# Do not edit - Assertion cell #
if mock.do_assertion():
    mock_env1 = deepcopy(mock.get_env())
    mock_env1.reset()
    mock_env2 = deepcopy(mock_env1)

    new_state, _, _, _ = mock_env1.step(0)
    assert(step(mock.get_Q(), mock_env2, 0, 0) == (0, -1, False, new_state)), \
    "Your environment did not return the correct values"
    print("Your step method seems to function properly, good job!")

## Create train method

Finally, we can create the method which will train our agent, i.e. update the Q-table.

### Task
In this method, you will enter your code at multiple places, but always between "Write code below" and "Write code above". 
- You will only input methods you have already created. 
- Remember to store the value in the right variable, i.e. the Q-table is stored in the variable Q_table
- To see which variables the methods returns, look at your implementation in the previous cells.
- PS. The following cells do not have assertion cells.

In [ ]:
def train(env, num_episodes, max_iterations):
    done = False
    epsilon = 1
    reward_list, iterations_list, epsilon_list = [], [], []
    
    "Write code below"
    # Create Q-table
    Q_table = create_q_table(env)
    "Write code above"

    for episode in range(num_episodes):

        if (done and episode % 10 == 0): 
            print("Episode: {} | Iterations: {} | Total Reward: {} | Epsilon: {}"\
                  .format(episode, iterations, total_reward, epsilon))
        
        state, done, iterations, total_reward = reset_env_and_update_params(env)
        
        "Write code below"
        # Update epsilon
        epsilon = update_epsilon(epsilon)
        
        "Write code above"
        
        while not done:
            
            "Write code below"
            
            # Take step
            action, reward, done, new_state = step(Q_table, env, state, epsilon)
                        
            # Update Q-table
            Q_table = update_q_table(Q_table, state, action, done, reward, new_state)
            
            "Write code above"
            
            # Set the new state as the current state
            state = new_state
            
            # Update episode information
            iterations += 1
            total_reward += reward
            
            # End episode if it has lasted for too many iterations
            if (iterations >= max_iterations): done = True

        reward_list.append(total_reward)
        iterations_list.append(iterations)
        epsilon_list.append(epsilon)
        
    return Q_table, (reward_list, iterations_list, epsilon_list)

## Create play method

This method can be used if you want to test the performance of your agent, by only running one episode.

In [ ]:
def play(env, Q_table):
    
    state, done, iterations, total_reward = reset_env_and_update_params(env)

    while not done:

        # Take step
        action, reward, done, new_state = step(Q_table, env, state, 0)

        # Set the new state as the current state
        state = new_state

        # Update episode information
        iterations += 1
        total_reward += reward

        if (iterations >= 100): done = True
    
    print("Iterations: {} | Total Reward: {}".format(iterations, total_reward))

## Time to put our methods to use!

Congratulation, you have created every method need to train your Q-learning agent!

### Task
To train your agent, simply set a value for the maximum_episodes and maximum_iterations
- Tip 1: Maximum_episodes should be less than 2000
- Tip 2: Maximum iterations should be less than 200

In [ ]:
maximum_episodes = 0
maximum_iterations = 0

Q_table_trained, data = train(env, maximum_episodes, maximum_iterations)

## Plot key variables 

This method plots different parameters during training. As a result, the training cell above must be run before this one.

### Task
- Simply run the cell below to create the plots

In [ ]:
try: 
    assert(data)
    plot_visualization(data)
except:
    print("You need to run the **train** cell before running this cell, as it uses the data from the training!")

## Play around for one round

This method runs one episode, using your trained Q-table. This is a quick way to test the performance.

### Task
- Run the cell below to quickly see the performance of your agent. This method does not alter your Q_table, so you can run it as many times as you like!

In [ ]:
play(env, Q_table_trained)

## Visualize an episode

### Task
This method works similarly to the one above, but visualizes the environment.
- If you would like to end the simulation, click on the Kernal tab in the toolbar, and click "Interrupt".

In [ ]:
render_performance(env, Q_table_trained)

## And we are done, almost!
Now you have everything you need to play around with the environment and alter different parameters to see how it impacts our agent. Can you improve the parameters already chosen? Suggestions:
- Increase the number of episodes and reduce iterations
- Change epsilon decay to change the Epsilon-Greedy Strategy
- Change the learning rate to make smaller or larger corrections to the Q-table per update
- Change the discount factor $\gamma$

If you would like to create your own cells, do the following:
1. Click on an existing cell
2. Click the Esc - button
3. Press "b" to add a cell below, press "a" the add a cell above, or double tap "d" to delete a cell

### Task
Run the cell below to turn off the assertion cells. <br>
Have fun!

In [ ]:
mock.turn_off_assertion()

## Want to try a different environment?
If you want to try a different environment suitable for Q-learning with tables, you can try this one:
- FrozenLake-v0
- You can read more about it here: https://gym.openai.com/envs/FrozenLake-v0/
- Tip: This environment may require the agent to do a more thorough exploration.

To use it, simply replace it with "Taxi-v2" in the cell where we created the env-variabel. 
Tip: Remember to rerun the cells.